In [1]:
import os
import warnings
from pyspark.sql import SparkSession

# 设置 JAVA_HOME
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@11"  # 替换为你的 Java 路径

# 初始化 SparkSession
spark = SparkSession.builder \
    .appName("Test PySpark") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/17 07:22:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
parquet_paths = [
    "./data/stock_prices_2000-2025.parquet",
]

In [3]:
df = spark.read.parquet(*parquet_paths)

In [4]:
# 创建临时视图以便使用 SQL 查询
df.createOrReplaceTempView("stock_price")

# 执行 SQL 查询
query = """
SELECT 
    year(dt) as year, count(*) as row_cnt, count(distinct symbol) as symbol_cnt
FROM stock_price 
--where adjust_type = 'hfq' and dt = '2023-01-10' and symbol = '000001'
group by year(dt)
order by year(dt)
limit 100;
"""

In [12]:
query = """
SELECT 
    *
FROM stock_price 
where 1=1
    and dt between '2023-01-09' and '2023-01-10'
    and symbol = '000001'
    -- adjust_type = 'hfq' 
limit 100;
"""


result = spark.sql(query)
result.show(50)

+----------+------+-------+-------+-------+-------+---------+---------------+---------+--------------+-------------+-------------+-----------+
|        dt|symbol|   open|  close|   high|    low|   volume|         amount|amplitude|change_percent|change_amount|turnover_rate|adjust_type|
+----------+------+-------+-------+-------+-------+---------+---------------+---------+--------------+-------------+-------------+-----------+
|2023-01-09|000001|2603.19|2611.31|2624.31| 2565.8|1057659.0| 1.5613684873E9|     2.27|          1.13|        29.25|         0.55|        hfq|
|2023-01-10|000001|2604.81| 2552.8|2625.94|2544.68|1269423.0|1.85109371676E9|     3.11|         -2.24|       -58.51|         0.65|        hfq|
|2023-01-09|000001|  14.75|   14.8|  14.88|  14.52|1057659.0| 1.5613684873E9|     2.46|          1.23|         0.18|         0.55|       none|
|2023-01-10|000001|  14.76|  14.44|  14.89|  14.39|1269423.0|1.85109371676E9|     3.38|         -2.43|        -0.36|         0.65|       none|